---
# DB 연결
---

In [ ]:
# -*- coding: utf-8 -*-

import mysql.connector
import pandas as pd 
import time

# MariaDB 연결 설정
connection = mysql.connector.connect(
    host="172.16.51.201",
    user="root",
    password="1231",
    database="crawling",
    port=3334,
    charset="utf8mb4",
    collation="utf8mb4_general_ci"
)
# 커서 생성
cursor = connection.cursor()

---
# 테이블 생성
---

In [ ]:
# 테이블 생성 쿼리
create_table_query = """
CREATE TABLE IF NOT EXISTS work24_data (
    id INT AUTO_INCREMENT PRIMARY KEY,
    company VARCHAR(255) NOT NULL,                        -- 훈련기관
    title VARCHAR(255) NOT NULL,                          -- 훈련명
    total_cost VARCHAR(50),                               -- 훈련비(책정 금액)
    cost_ VARCHAR(50),                                    -- 훈련비(일반 훈련생)
    job_acceptance_rate VARCHAR(50),                      -- 훈련기관 직종별 취업률
    average_score VARCHAR(50),                            -- 수강생 평균 만족도
    NCS_classification VARCHAR(255),                      -- NCS 직무분류
    NCS_level VARCHAR(10),                                -- NCS 수준
    NCS_application_status VARCHAR(10),                   -- NCS 적용 여부
    certifications VARCHAR(255),                          -- 관련 자격증
    training_duration VARCHAR(100),                       -- 훈련 기간
    training_time VARCHAR(50),                            -- 훈련 시간
    average_age VARCHAR(50),                              -- 평균 연령대
    Pname VARCHAR(100),                                   -- 담당자 성명
    Pphone VARCHAR(50),                                   -- 담당자 번호
    Pmail VARCHAR(100),                                   -- 담당자 메일
    depart VARCHAR(255),                                  -- 주관부처
    train VARCHAR(255),                                   -- 훈련유형
    day_night_weekend VARCHAR(300),                       -- 주야구분/주말여부
    training_goal TEXT,                                   -- 훈련목표
    train_type VARCHAR(100),                              -- 훈련내용
    average_satisfaction VARCHAR(50),                     -- 평균만족도
    number_of_students VARCHAR(50),                       -- 수강인원
    number_of_evaluators VARCHAR(50),                     -- 평가인원
    evaluation_participation_rate VARCHAR(50),            -- 평가참가율
    review TEXT,                                          -- 후기
    recommend VARCHAR(50) DEFAULT '-',                                -- 추천합니다
    overall_satisfaction VARCHAR(50) DEFAULT '-',                     -- 전반적 만족도
    career_development_support VARCHAR(50) DEFAULT '-',               -- 경력개발지원
    assignments_and_feedback VARCHAR(50) DEFAULT '-',                 -- 과제 및 피드백
    instructors VARCHAR(50) DEFAULT '-',                              -- 교강사
    training_program_planning_and_operation VARCHAR(50) DEFAULT '-',  -- 훈련과정 편성 및 운영
    training_program_field_orientation VARCHAR(50) DEFAULT '-',       -- 훈련과정 현장지향성
    training_environment_and_equipment VARCHAR(50) DEFAULT '-',       -- 훈련환경 및 장비
    skill_improvement VARCHAR(50) DEFAULT '-',                        -- 능력향상
    goal_achievement VARCHAR(50) DEFAULT '-',                         -- 목표달성
    value_of_training VARCHAR(50) DEFAULT '-',                        -- 수강가치
    project_based_learning VARCHAR(255) DEFAULT '-',                  -- 프로젝트 학습
    training_program_practical_connection VARCHAR(255) DEFAULT '-',   -- 훈련과정 실무연계성
    training_content VARCHAR(255) DEFAULT '-',                        -- 훈련내용
    training_instructors VARCHAR(255) DEFAULT '-',                    -- 훈련교사
    training_method VARCHAR(255) DEFAULT '-',                         -- 훈련방법
    facilities_and_equipment VARCHAR(255) DEFAULT '-',                -- 시설장비
    administrative_services VARCHAR(255) DEFAULT '-',                 -- 행정서비스
    employment_support VARCHAR(255) DEFAULT '-',                      -- 취업지원
    training_level VARCHAR(255) DEFAULT '-',                          -- 훈련수준(3점)
    training_achievement VARCHAR(255) DEFAULT '-',                    -- 훈련성취도(4점)
    employment_possibility VARCHAR(255) DEFAULT '-',                  -- 취업가능성
    created_at TIMESTAMP DEFAULT (CURRENT_TIMESTAMP + INTERVAL 9 HOUR)   -- 데이터 삽입 시간
);
"""

# 테이블 생성 실행
try:
    cursor.execute(create_table_query)
    print("테이블 'work24_data'가 성공적으로 생성되었습니다.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

# 변경 사항 커밋
connection.commit()

---
# 데이터 조회/삭제
---

In [ ]:
delete_query = "DELETE FROM work24_data;"
delete_table_query = "DROP TABLE IF EXISTS work24_data;"
cursor.execute(delete_table_query)

# 변경 사항 커밋
connection.commit()

---
# 데이터 삽입
---

In [ ]:
df = df.fillna('-')
# 컬럼명을 리스트로 가져오기
columns = df.columns.tolist()

# SQL INSERT 쿼리 생성
insert_query = f"""
    INSERT INTO work24_data ({', '.join(columns)})
    VALUES ({', '.join(['%s'] * len(columns))})
"""

# 데이터 삽입
for i, row in df.iterrows():
    values = tuple(row)
    cursor.execute(insert_query, values)

# 변경 사항 커밋
connection.commit()

---
# db 데이터 df로 가져오기
---

In [ ]:
# 데이터 조회 쿼리
from_table_query = "SELECT * FROM work24_data;"
cursor.execute(from_table_query)
# 결과 가져오기
result = cursor.fetchall()

# 컬럼 이름 가져오기
columns = [desc[0] for desc in cursor.description]

# 데이터프레임으로 변환
db_df = pd.DataFrame(result, columns=columns)

db_df

---
# 연결 종료
---

In [ ]:
# 연결 종료
cursor.close()
connection.close()

---
# 기업별 크롤링
---

In [39]:
import mysql.connector
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoAlertPresentException, UnexpectedAlertPresentException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.alert import Alert
import re
import time
from datetime import datetime
import pandas as pd
import logging

# 기본 로거 설정
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# 파일 핸들러 설정
file_handler = logging.FileHandler('app.log', encoding='utf-8')  # UTF-8 인코딩 설정
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

# 핸들러 추가
logger.addHandler(file_handler)

# 현재 시간 출력 
now = datetime.now()
# 현재 시간을 출력하기
print("▶️▶️▶️▶️▶️▶️▶️▶️▶️▶️ 시작 시간 :", now, "◀️◀️◀️◀️◀️◀️◀️◀️◀️◀️")

def connect_to_db():
    return mysql.connector.connect(
        host="172.16.51.201",
        user="root",
        password="1231",
        database="crawling",
        port=3334,
        charset="utf8mb4",
        collation="utf8mb4_general_ci"
    )

def setup_driver():
    options = Options()
    # options.add_argument("--headless")  # 헤드리스 모드 추가
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-popup-blocking")  # 팝업 차단 옵션
    driver = webdriver.Chrome(options=options, service=Service(ChromeDriverManager().install()))
    return driver

def scrape_data(driver, keyword):
    df = pd.DataFrame()
    start_date = 20220101
    url = f'https://www.work24.go.kr/hr/a/a/1100/trnnCrsInf0.do?endDate=20251023&keyword1=&keyword2=&pageSize=10&orderBy=&startDate_datepicker=2024-10-23&currentTab=1&topMenuYn=&pop=&crseTracseSeNum=&keywordType=1&gb=&keyword={keyword}&kDgtlYn=&area=00%7C%EC%A0%84%EA%B5%AD+%EC%A0%84%EC%B2%B4&orderKey=&mberSe=&kdgLinkYn=&tranRegister=&mberId=&searchYn=Y&pageId=&programMenuIdentification=EBG020000510010&endDate_datepicker=2025-10-23&monthGubun=&pageIndex=1&bgrlInstYn=&tracseTab=1&startDate={start_date}&crseTracseSe=&ncs=&gvrnInstt=&selectNCSKeyword=&requestURI=%2Fhr%2Fa%2Fa%2F1100%2FtrnnCrsInf0Post.do&action=trnnCrsInf0Post.do'
    driver.get(url)

    time.sleep(5)  # 페이지 시도 전 대기 시간

    for i in range(1, 11):
        try:
            # Data Extraction
            type_xpath = f'//*[@id="tab-panel-02"]/div/div[1]/div[2]/div[{i}]/div[2]/p[1]/span'
            type_element = driver.find_element(By.XPATH, type_xpath)
            type_ = re.sub(r'\n', ', ', type_element.text)
            time.sleep(3) 
            driver.find_element(By.XPATH, f'//*[@id="tab-panel-02"]/div/div[1]/div[2]/div[{i}]/div[2]/h3/a[2]').click()
            time.sleep(3)
            driver.switch_to.window(driver.window_handles[-1])
            
            # Scrape information
            company_ = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[1]/p').text
            title_ = re.sub(r'(모집마감|모집중)', '', driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[1]/h4').text).strip()
            total_cost_ = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[2]/dl/dd[1]/p').text
            cost_ = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[2]/dl/dd[1]/div/p').text
            percent_ = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[2]/div/ul/li[1]/div/div/span').text
            ALLscore_text = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[2]/div/ul/li[2]/span[2]/span').get_attribute("title")
            ALLscore_ = float(re.sub('[가-힣]','', re.search(r'기준(\d+(\.\d+)?)점', ALLscore_text).group()))
            NCS_job_ = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[2]/div/ul/li[3]/span[2]').text
            NCS_level_ = re.sub('[^0-9]','', driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[2]/div/ul/li[4]/span[2]').text)
            test_ = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[2]/div/ul/li[5]/span[2]').text.split()[0]
            certi_ = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[2]/div/ul/li[6]/span[2]').text
            date_ = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[2]/div/ul/li[7]/span[2]').text
            time_ = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[2]/div/ul/li[8]/span[2]').text
            old_ = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[2]/div/ul/li[9]/span[2]').text
            Pname_ = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[2]/div/ul/li[10]/span[2]').text
            Pphone_ = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[2]/div/ul/li[11]/span[2]').text
            Pmail_ = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[2]/div/ul/li[12]/span[2]').text
            depart_ = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[2]/div/ul/li[13]/span[2]').text
            train_ = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[2]/div/ul/li[14]/span[2]').text
            # 먼저 첫 번째 XPath에서 텍스트를 확인
            key_text = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[2]/div/ul/li[16]/span[1]').text

            # "주야구분/주말여부" 텍스트를 확인한 후 조건 분기
            if key_text == "주야구분/주말여부":
                # 주야구분/주말여부일 경우 li[16]/span[2]의 텍스트 추출
                night_ = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[2]/div/ul/li[16]/span[2]').text
            else:
                # 그렇지 않으면 li[17]/span[2]의 텍스트 추출
                night_ = driver.find_element(By.XPATH, '//*[@id="section1"]/div/div[2]/div[1]/div[2]/div/ul/li[17]/span[2]').text
            skill_ = driver.find_element(By.XPATH, '//*[@id="section1-1"]/div[2]/table/tbody').text

            # 후기 정보
            driver.find_element(By.XPATH, '//*[@id="infoTab4"]/button').click()
            time.sleep(1)
            result_elements_1 = driver.find_elements(By.CLASS_NAME, 'item')
            hoo_row = {}

            for result_score in result_elements_1:
                result_text = result_score.text
                key__ = result_text.split('\n')[0]
                key_ = re.sub(r'\((3|4|5)점\)', '', key__)

                # 첫 번째 형식인지 두 번째 형식인지 판별
                if len(result_text.split('\n')) > 1:
                    value__ = result_text.split('\n')[1]
                else:
                    value__ = result_text.split(' ')[-1]
                try:
                    if key_ != '추천합니다':
                        value_ = re.sub(r'[^0-9.]','',value__.split('\n')[1])
                    else:
                        value_ = value__.split('\n')[1]
                except:
                    value_ = value__
                # hoo_row에 key-value 저장
                hoo_row[f'{key_}'] = value_

            # 후기 내용
            try:
                try:
                    # 첫 번째 XPATH 시도
                    review_text = driver.find_element(By.XPATH, '//*[@id="section1-4"]/div[8]').text
                except:
                    # 첫 번째 XPATH가 없을 경우 두 번째 XPATH 시도
                    review_text = driver.find_element(By.XPATH, '//*[@id="section1-4"]/div[7]').text

                # 성공적으로 찾았을 경우 저장
                hoo_row['review'] = review_text
            except Exception as e:
                # 둘 다 없을 경우 except로 넘어감
                print(f"Error retrieving review: {e}")

            # 빈 값이 있는 항목 필터링(상세 만족도가 없는 경우 빈 딕셔너리를 생성함)
            hoo_row = {k: v for k, v in hoo_row.items() if k and v}
            # Create row
            row_ = {'company' : company_, 'title' : title_,  'total_cost' : total_cost_, 'cost_' : cost_, 
                    'job_acceptance_rate' : percent_, 'average_score' : ALLscore_, 'NCS_classification' : NCS_job_,
                    'NCS_level' : NCS_level_, 'NCS_application_status' : test_, 'certifications' : certi_, 'training_duration' : date_,
                    'training_time' : time_, 'average_age' : old_, 'Pname' : Pname_, 'Pphone' : Pphone_,
                    'Pmail' : Pmail_, 'depart' : depart_, 'train' : train_, 'day_night_weekend' : night_, 
                    'training_goal' : skill_, 'train_type' : type_}
            row_.update(hoo_row)

            df = pd.concat([df, pd.DataFrame([row_])], ignore_index=True)
            df.reset_index(drop=True, inplace=True)
            
            driver.close()
            # time.sleep(4)
            driver.switch_to.window(driver.window_handles[0])    
        
        except NoSuchElementException:
            print(f"No more elements to scrape at index {i}. Exiting loop.")
            driver.close()  # 더 이상 요소가 없을 때 루프 종료
            break

            
    return df

def save_to_db(df, cursor, connection):
    column_mapping = {
        '평균만족도': 'average_satisfaction',
        '수강인원': 'number_of_students',
        '평가인원': 'number_of_evaluators',
        '평가참가율': 'evaluation_participation_rate',
        'review': 'review',
        '추천합니다': 'recommend',
        '전반적 만족도': 'overall_satisfaction',
        '경력개발지원': 'career_development_support',
        '과제 및 피드백': 'assignments_and_feedback',
        '교강사': 'instructors',
        '훈련과정 편성 및 운영': 'training_program_planning_and_operation',
        '훈련과정 현장지향성': 'training_program_field_orientation',
        '훈련환경 및 장비': 'training_environment_and_equipment',
        '훈련환경': 'training_environment',
        '능력향상': 'skill_improvement',
        '목표달성': 'goal_achievement',
        '수강가치': 'value_of_training',
        '프로젝트 학습': 'project_based_learning',
        '훈련과정 실무연계성': 'training_program_practical_connection',
        '훈련내용': 'training_content',
        '훈련교사': 'training_instructors',
        '훈련방법': 'training_method',
        '시설장비': 'facilities_and_equipment',
        '행정서비스': 'administrative_services',
        '취업지원': 'employment_support',
        '훈련수준': 'training_level',
        '학습지원': 'training_support',        
        '훈련성취도': 'training_achievement',
        '취업가능성': 'employment_possibility',
        '평가 및 피드백' : 'test_and_feedback',
        '고용가능성' : 'employability'
    }
    
    df.rename(columns=column_mapping, inplace=True)
    df = df.fillna('-')
    
    columns = df.columns.tolist()
    insert_query = f"""
        INSERT INTO work24_data ({', '.join(columns)})
        VALUES ({', '.join(['%s'] * len(columns))})
    """
    
    for i, row in df.iterrows():
        cursor.execute(insert_query, tuple(row))
    connection.commit()

# 팝업 닫기 함수
def close_alert(driver):
    try:
        alert = Alert(driver)
        alert.dismiss()  # 팝업을 닫음
        logging.info("팝업이 닫혔습니다.")
    except NoAlertPresentException:
        logging.info("팝업이 존재하지 않습니다.")

▶️▶️▶️▶️▶️▶️▶️▶️▶️▶️ 시작 시간 : 2024-10-23 10:09:51.865572 ◀️◀️◀️◀️◀️◀️◀️◀️◀️◀️


In [40]:
def main():
    connection = connect_to_db()
    cursor = connection.cursor()
    driver = setup_driver()
    start_time = datetime.now()

    try:
        keyword = '네이버클라우드'  # 단일 키워드
        logging.info(f"크롤링 시작 시간: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

        try:
            logging.info(f" {keyword} 시작")              
            df = scrape_data(driver, keyword)

            if df.empty:
                logging.info(f" {keyword}에 데이터가 없습니다. 종료합니다.")
            else:
                save_to_db(df, cursor, connection)
                logging.info(f" {keyword} 데이터베이스에 저장 완료")
                logging.info(f" {keyword} 종료")

        except UnexpectedAlertPresentException as e:
            logging.error(f"Unexpected {keyword}: {str(e)}")
            close_alert(driver)  # 팝업을 닫음
            time.sleep(5)  # 5초 대기 후 다시 시도

    finally:
        driver.quit()
        cursor.close()
        connection.close()
        end_time = datetime.now()
        elapsed_time = end_time - start_time
        logging.info(f"크롤링 종료 시간: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
        logging.info(f"총 소요 시간: {elapsed_time}")

if __name__ == "__main__":
    main()

No more elements to scrape at index 3. Exiting loop.


In [6]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [41]:
df = pd.read_csv(r"C:\Users\user\work24_data_202410231017.csv")

df.to_excel(r"C:\Users\user\work24_data_202410231017.xlsx")